In [42]:
import os
import pydicom
import numpy as np
import SimpleITK as sitk

## 获取同一ct的dcm文件数量

In [16]:
def get_nii(path,tmp=18):
    a = os.listdir(path)
    b = {}
    for i in a:
        if i[:tmp] in b:
            b[i[:tmp]] += 1
        else:
            b[i[:tmp]] = 1
    return b

In [17]:
jiang = get_nii("./data/JIANG_LIN_QUAN-CT0931273G/",18)

In [18]:
lin = get_nii("./data/LIN_JIU_WANG-CT1152288E/",len("LIN_JIU_WANG_01"))

In [20]:
linq = get_nii("./data/LinQingNian-BHSZ1322B/",len("LinQingNian_01"))

In [22]:
yan = get_nii("./data/YAN_ZI_BI-CT1083097E/",len("YAN_ZI_BI_01"))

In [24]:
zhang = get_nii("./data/ZHANG_AI_XIANG-CT1112240C/",len("ZHANG_AI_XIANG_01"))

In [38]:
d = {}
d.update(jiang)
d.update(lin)
d.update(linq)
d.update(yan)
d.update(zhang)

In [39]:
d

{'JIANG_LIN_QUAN_204': 121,
 'JIANG_LIN_QUAN_202': 275,
 'JIANG_LIN_QUAN_201': 55,
 'JIANG_LIN_QUAN_203': 55,
 'JIANG_LIN_QUAN_301': 4,
 'JIANG_LIN_QUAN_100': 1,
 'LIN_JIU_WANG_30': 318,
 'LIN_JIU_WANG_02': 92,
 'LIN_JIU_WANG_01': 1,
 'LIN_JIU_WANG_99': 1,
 'LinQingNian_02': 47,
 'LinQingNian_03': 332,
 'LinQingNian_04': 58,
 'LinQingNian_01': 2,
 'LinQingNian_50': 1,
 'YAN_ZI_BI_04': 236,
 'YAN_ZI_BI_03': 41,
 'YAN_ZI_BI_02': 41,
 'YAN_ZI_BI_05': 94,
 'YAN_ZI_BI_01': 1,
 'YAN_ZI_BI_90': 2,
 'ZHANG_AI_XIANG_04': 257,
 'ZHANG_AI_XIANG_06': 97,
 'ZHANG_AI_XIANG_05': 43,
 'ZHANG_AI_XIANG_03': 44,
 'ZHANG_AI_XIANG_90': 2,
 'ZHANG_AI_XIANG_01': 1}

In [36]:
data_list = os.listdir("./data/")


In [40]:
# filter less than 10 dcm and copy to dataset
for key,values in d.items():
    if values > 10:
        path = os.path.join("./dataset/",key)
        if not os.path.exists(path):
            os.mkdir(path)
        for i in data_list:
            if key[:6] in i:
                data_path = os.path.join("./data/",i)
                cmd = f"cp -f {data_path}/{key}* {path}"
                os.system(cmd)

## 生成3维nii图像文件

In [46]:
def gen_nii(dataset_path="./dataset/",nii_path="./nii"):
    dcm_list = os.listdir(dataset_path)
    for dcm in dcm_list:
        # dcm path
        dcm_path = os.path.join(dataset_path,dcm)

        # Ensure output directory exists
        os.makedirs(nii_path, exist_ok=True)

        # Load DICOM files
        dicom_files = sorted([os.path.join(dcm_path, filename) for filename in os.listdir(dcm_path) if filename.endswith(".dcm")])
        dicom_slices = [pydicom.dcmread(file) for file in dicom_files]

        # Sort slices by slice position (z-coordinate)
        dicom_slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

        # Extract pixel data and convert to a 3D numpy array
        slice_shape = dicom_slices[0].pixel_array.shape
        ct_volume = np.zeros((len(dicom_slices), slice_shape[0], slice_shape[1]), dtype=np.int16)

        for i, ds in enumerate(dicom_slices):
            ct_volume[i, :, :] = ds.pixel_array

        # Convert pixel spacing to a list of floats
        pixel_spacing = [float(dicom_slices[0].PixelSpacing[0]), float(dicom_slices[0].PixelSpacing[1]), float(dicom_slices[0].SliceThickness)]

        # Create SimpleITK image
        ct_image = sitk.GetImageFromArray(ct_volume)
        ct_image.SetSpacing(pixel_spacing)

        # Save as .nii file
        nii_filename = os.path.join(nii_path, f"{dcm}.nii")
        sitk.WriteImage(ct_image, nii_filename)

        print(f"CT image saved as {nii_filename}, Shape is ",ct_volume.shape)

In [47]:
gen_nii()

CT image saved as ./nii/YAN_ZI_BI_05.nii, Shape is  (94, 512, 512)
CT image saved as ./nii/LIN_JIU_WANG_02.nii, Shape is  (92, 512, 512)
CT image saved as ./nii/LinQingNian_02.nii, Shape is  (47, 512, 512)
CT image saved as ./nii/LinQingNian_03.nii, Shape is  (332, 512, 512)
CT image saved as ./nii/YAN_ZI_BI_04.nii, Shape is  (236, 512, 512)
CT image saved as ./nii/ZHANG_AI_XIANG_06.nii, Shape is  (97, 512, 512)
CT image saved as ./nii/JIANG_LIN_QUAN_202.nii, Shape is  (275, 512, 512)
CT image saved as ./nii/JIANG_LIN_QUAN_204.nii, Shape is  (121, 512, 512)
CT image saved as ./nii/ZHANG_AI_XIANG_05.nii, Shape is  (43, 512, 512)
CT image saved as ./nii/YAN_ZI_BI_02.nii, Shape is  (41, 512, 512)
CT image saved as ./nii/JIANG_LIN_QUAN_201.nii, Shape is  (55, 512, 512)
CT image saved as ./nii/ZHANG_AI_XIANG_03.nii, Shape is  (44, 512, 512)
CT image saved as ./nii/LinQingNian_04.nii, Shape is  (58, 512, 512)
CT image saved as ./nii/ZHANG_AI_XIANG_04.nii, Shape is  (257, 512, 512)
CT image s

## 利用nii文件生成drr图像
- 利用ctdrr程序

In [ ]:
x = np.lins